Following this [crash course](https://towardsdatascience.com/convolutional-neural-networks-with-tensorflow-2d0d41382d32) here

In [1]:
# from tensorflow import keras

# kerasCachePath = os.path.abspath(os.path.join(os.getcwd(), os.pardir,".keras"))

# async def get_data ():
#     await keras.utils.get_file(
#             fname='cats_dogs',
#             origin='https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip',
#             cache_dir=kerasCachePath,
#             extract=True
#         )

#     base_dir = kerasCachePath

#     train_dir = os.path.join(base_dir, 'train')
#     validation_dir = os.path.join(base_dir, 'validation')

#     train_cats_dir = os.path.join(train_dir, 'cats')
#     train_dogs_dir = os.path.join(train_dir, 'dogs')
#     train_cats_fnames = os.listdir(train_cats_dir)
#     train_dog_fnames = os.listdir(train_dogs_dir)

#     validate_cats_dir = os.path.join(validation_dir, 'cats')
#     validate_dogs_dir = os.path.join(validation_dir, 'dogs')
#     validate_cats_fnames = os.listdir(validate_cats_dir)
#     validate_dog_fnames = os.listdir(validate_dogs_dir)


# await get_data()

In [2]:
import os
base_dir = os.path.abspath(
    os.path.join(os.getcwd(), os.pardir, 'datasets/cats_and_dogs_filtered')
)

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')
train_cats_fnames = os.listdir(train_cats_dir)
train_dog_fnames = os.listdir(train_dogs_dir)

validate_cats_dir = os.path.join(validation_dir, 'cats')
validate_dogs_dir = os.path.join(validation_dir, 'dogs')
validate_cats_fnames = os.listdir(validate_cats_dir)
validate_dog_fnames = os.listdir(validate_dogs_dir)

train_cat_fnames = os.listdir(train_cats_dir)
# print(train_cat_fnames[:10])

train_dog_fnames = os.listdir(train_dogs_dir)
train_dog_fnames.sort()
# print(train_dog_fnames[:10])

print("Count of training cat images:", len(train_cats_fnames))
print("Count of training dog images:", len(train_dog_fnames))
print("Count of validation cat images:", len(validate_cats_fnames))
print("Count of validation dog images:", len(validate_dog_fnames))

Count of training cat images: 1000
Count of training dog images: 1000
Count of validation cat images: 500
Count of validation dog images: 500


In [3]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

imgPath = os.path.join(train_cats_dir, train_cats_fnames[0])
img = mpimg.imread(imgPath)
flatImg = img.flatten()
print("Each image has " + str(img.shape[2]) + " channels, each within the range of " + str(flatImg.min()) + " and " + str(flatImg.max()))

Each image has 3 channels, each within the range of 0 and 255


In [4]:
from tensorflow.keras import Sequential, layers, optimizers, losses, metrics
model = Sequential()

model.add(layers.Conv2D(filters=16, kernel_size=(3,3), activation='relu', input_shape=(150,150,3)))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))# 512 neuron hidden layer
model.add(layers.Dense(1, activation='sigmoid')) # 1 output because it is either Cat or Dog

model.compile(optimizer=optimizers.RMSprop(learning_rate=0.001), loss=losses.BinaryCrossentropy(), metrics=[metrics.Accuracy()] )
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 18496)             0

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255 so that we normalize each pixel between 0 and 1
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow validation images in batches of 20 using val_datagen generator
validation_generator = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [10]:
history = model.fit(
      train_generator,
      steps_per_epoch=100,  # 2000 images = batch_size * steps
      epochs=15,
      validation_data=validation_generator,
      validation_steps=50,  # 1000 images = batch_size * steps
      verbose=2)

Epoch 1/15
100/100 - 25s - loss: 0.0378 - accuracy: 0.1915 - val_loss: 1.9039 - val_accuracy: 0.0930
Epoch 2/15
100/100 - 24s - loss: 0.0290 - accuracy: 0.1820 - val_loss: 1.9370 - val_accuracy: 0.0940
Epoch 3/15
100/100 - 24s - loss: 0.0330 - accuracy: 0.2015 - val_loss: 1.8802 - val_accuracy: 0.0890
Epoch 4/15
100/100 - 24s - loss: 0.0351 - accuracy: 0.2160 - val_loss: 2.0689 - val_accuracy: 0.0850
Epoch 5/15
100/100 - 24s - loss: 0.0229 - accuracy: 0.2075 - val_loss: 2.1323 - val_accuracy: 0.1100
Epoch 6/15
100/100 - 24s - loss: 0.0115 - accuracy: 0.2475 - val_loss: 2.7080 - val_accuracy: 0.1660
Epoch 7/15
100/100 - 24s - loss: 0.0364 - accuracy: 0.3040 - val_loss: 2.4264 - val_accuracy: 0.0980
Epoch 8/15
100/100 - 24s - loss: 0.0133 - accuracy: 0.2625 - val_loss: 2.6355 - val_accuracy: 0.1590
Epoch 9/15
100/100 - 24s - loss: 0.0292 - accuracy: 0.3060 - val_loss: 2.6952 - val_accuracy: 0.1150
Epoch 10/15
100/100 - 24s - loss: 0.0349 - accuracy: 0.3040 - val_loss: 2.4236 - val_accura

In [29]:
import matplotlib.pyplot as plt

cat = plt.imread('../datasets/testKitty.jpeg')

print(cat.shape)
# model.predict(cat)
model.predict(np.array([cat,]))

(183, 275, 3)


ValueError: in user code:

    /Users/jontan_paloit/Dev/Jonalogy/PythonNest/workingUpToMyFirstCNN/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1462 predict_function  *
        return step_function(self, iterator)
    /Users/jontan_paloit/Dev/Jonalogy/PythonNest/workingUpToMyFirstCNN/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1452 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/jontan_paloit/Dev/Jonalogy/PythonNest/workingUpToMyFirstCNN/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/jontan_paloit/Dev/Jonalogy/PythonNest/workingUpToMyFirstCNN/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/jontan_paloit/Dev/Jonalogy/PythonNest/work